In [79]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [80]:
os.environ['GROQ_API_KEY']=os.getenv("GROQ_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ['HF_TOKEN']=os.getenv("huggingface_api")


In [81]:
from langchain_community.document_loaders import PyPDFLoader
# loader = PyPDFLoader("Data/sample-invoice.pdf")
loader = loader = PyPDFLoader("K:/RAG/Data/sample-invoice.pdf")
document = loader.load()
document

[Document(metadata={'producer': 'Microsoft® Excel® 2016', 'creator': 'Microsoft® Excel® 2016', 'creationdate': '2023-09-08T15:57:51+02:00', 'author': 'CPB Software (Germany) GmbH', 'keywords': '"WMACCESS, Invoice, Sample Invoice"', 'moddate': '2023-09-08T15:59:22+02:00', 'subject': 'WMACCESS', 'title': 'Sample Invoice', 'source': 'K:/RAG/Data/sample-invoice.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='CPB Software (Germany) GmbH - Im Bruch 3 - 63897 Miltenberg/Main\nMusterkunde AG\nMr. John Doe\nMusterstr. 23\n12345 Musterstadt Name:  Stefanie Müller\nPhone: +49 9371 9786-0\nInvoice WMACCESS Internet\nVAT No. DE199378386\nInvoice No\n123100401\nAmount\n-without VAT- quantity\n130,00 € 1\n10,00 € 0\n50,00 € 0\n1.000,00 € 0\n10,00 € 0\n0,58 € 14\n0,70 € 0\n1,50 € 162\n0,50 € 0\n0,80 € 0\n1,80 € 0\n0,30 € 0\n0,30 € 0\n0,40 € 0\n0,40 € 0\n0,30 € 0\n0,30 € 0\nTerms of Payment: Immediate payment without discount. Any bank charges must be paid by the invoice recipient.

In [82]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
data_base = FAISS.from_documents(document,embeddings)

In [83]:
print(data_base)


In [84]:
retriever = data_base.as_retriever(search_kwargs={"k": 2})

In [85]:
# from langchain.chat_models import ChatGroq
from langchain_groq import ChatGroq

llm =ChatGroq(model="llama-3.3-70b-versatile",api_key=groq_api_key) 

In [86]:

invoice_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
You are an intelligent assistant specialized in understanding and analyzing invoice documents.

Use the following context to answer the question. Focus on:
- Invoice number, customer number, date
- Name and phone
- Invoice period
- Service description
- Amount without VAT
- VAT number
- Total amounts
- Gross amount including VAT

Respond clearly and professionally. If information is missing, say so without guessing.

Context:
{context}

Question:
{question}
"""
)






In [87]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": invoice_prompt},
    return_source_documents=False
)
"What services were provided?
query ="What services were provided? "
response = rag_chain.invoke(query)

print(response["result"])


Object `provided` not found.
Based on the provided invoice document, the services that were provided can be inferred from the "Service Description" section, although it appears to be partially incomplete or not fully detailed. However, we can identify the following services and their associated amounts without VAT:

1. **Basic Fee wmPos** - No amount specified.
2. **Change of user accounts** - No amount specified.
3. **Transaction Fee T6** - 0.00 €
4. **Basis fee for additional user accounts** - 0.00 €
5. **Basic Fee wmView** - No amount specified.
6. **Basic Fee wmGuide** - No amount specified.

Additionally, there are various transaction fees mentioned (T1 to T6 and G1 to G6), but most of them have a value of 0.00 €, except for:

- **T1**: Related to a charge of 0.58 € per unit, with 14 units charged, totaling 8.12 €.
- **T3**: Related to a charge of 1.50 € per unit, with 162 units charged, totaling 243.00 €.

These services are associated with specific rates and quantities, as detai